# Get Stock Data

# PROBLEM: Tickers ARE WRONG

In [19]:
import datetime as dt

import eikon as ek
import pandas as pd
from pandas.errors import MergeError
import pandas_datareader.data as web
#from pandas_datareader._utils import RemoteDataError
from tqdm.auto import tqdm

from utilities import columns, filter_dataset

C:\Users\Lorenzo\miniconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [9]:
with open('key.secret') as file:
    KEY = file.read()
FILENAME = 'cleaned_green_bonds.csv'

In [10]:
try:
    ek.set_app_key(KEY)
    bonds, err = ek.get_data(['CN151819SH='], 'TR.FirstAnnounceDate')
    # sanity check to prevent silent error
    sanity_check = bonds['First Announcement Date'][0]
    if sanity_check != '2019-07-11':
        raise ValueError(f'Sanity check not passed: {sanity_check}')
except ek.EikonError:
    raise ValueError('Please check key.secret, then run Eikon Desktop or Eikon API Proxy')

In [4]:
FILENAME = 'cleaned_green_bonds.csv'
df = pd.read_csv(FILENAME,
                 parse_dates=['Issue Date', 'First Announcement Date'],
                 dtype={'Ultimate Parent ID': str})

# only public companies
df = df[df['Organization Is Public Flag'] == True]
df = filter_dataset.filter_dataset(df)

Excluded 0 rows over 456. Green bonds before 2012 must be due to errors and 2020 might pollute data due to COVID and year not finished yet.


In [102]:
df.columns

Index(['Preferred RIC', 'Issuer', 'Ticker', 'Coupon', 'Maturity', 'Issue Date',
       'ISIN', 'Principal Currency', 'Country of Issue', 'Issuer Type',
       'Instrument Type', 'Amount Issued (USD)', 'Green Bond', 'Sector',
       'TRBC Sector', 'Country of Incorporation', 'SIC', 'Use of Proceeds',
       'PBOC Bond Rating', 'Current Coupon Class', 'Instrument',
       'First Announcement Date', 'Issuer Rating',
       'Organization Is Public Flag', 'Parent Long Name', 'Ultimate Parent Id',
       'Coupon Class Description', 'IsParentPublic'],
      dtype='object')

In [25]:
pivot_column = 'Country of Incorporation'

country_count = pd.pivot_table(df[[pivot_column, 'Ticker']],
               index=[pivot_column],
               aggfunc='count')
country_count

,Ticker
Country of Incorporation,
Australia,11
Austria,7
Belgium,1
Bermuda,5
Brazil,10
Canada,6
Cayman Islands,6
Chile,4
China (Mainland),55


In [31]:
country_count['Ticker'].index

Index(['Australia', 'Austria', 'Belgium', 'Bermuda', 'Brazil', 'Canada',
       'Cayman Islands', 'Chile', 'China (Mainland)', 'Colombia', 'Denmark',
       'Finland', 'France', 'Germany', 'Hong Kong', 'India', 'Italy', 'Japan',
       'Lithuania', 'Luxembourg', 'Mexico', 'Morocco', 'Netherlands',
       'New Zealand', 'Nigeria', 'Norway', 'Peru', 'Philippines', 'Portugal',
       'Singapore', 'South Africa', 'South Korea', 'Spain', 'Sweden',
       'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'United Arab Emirates',
       'United Kingdom', 'United States'],
      dtype='object', name='Country of Incorporation')

In [65]:
country_to_index = {'Australia': '^AXJO',
                    'Austria': '^ATX',
                    'Belgium': '^BFX',
                    #'Bermuda': '^BSX',
                    'Brazil': '^BVSP',
                    'Canada': '^GSPTSE',
                    # 'Cayman Islands': None, # ?????
                    #'Chile': '^IPSA',
                    'China (Mainland)': '000001.SS',  # ???
                    #'Colombia': '^SPCOLCSP',
                    #'Denmark': '^OMXC20',
                    'Finland': '^OMXH25',
                    'France': '^FCHI',
                    'Germany': '^GDAXI',
                    'Hong Kong': '^HSI',
                    'India': '^BSESN',
                    #'Italy': 'FTSEMIB.MI',
                    'Japan': '^N225',
                    'Lithuania': '^OMXVGI',
                    #'Luxembourg': 'MCXNUK.L',
                    'Mexico': '^MXX',
                    #'Morocco': '^NQMAEUR',  # ??
                    'Netherlands': '^AEX',
                    'New Zealand': '^NZ50',
                    # 'Nigeria', # ??????
                    #'Norway': 'OBX.OL',
                    'Peru': '^SPBLPGPT',
                    'Philippines': 'PSEI.PS',
                    #'Portugal': 'PSI20.LS',
                    #'Singapore': '^STI',
                    'South Africa': 'JSEJF',
                    'South Korea': '^KS11',
                    'Spain': '^IBEX',
                    'Sweden': '^OMX',
                    'Switzerland': '^SSMI',
                    'Taiwan': '^TWII',
                    #'Thailand': '^SET.BK',
                    #'Turkey': 'XU100.IS',
                    'United Arab Emirates': 'UAE',
                    #'United Kingdom': 'FT30.L',
                    'United States': '^GSPC'
                    }

In [70]:
web.DataReader('^FTSE', 'yahoo', '2019-10-1', '2020-10-1')

RemoteDataError: No data fetched for symbol ^FTSE using YahooDailyReader

In [66]:
for k, v in country_to_index.items():
    print(k)
    web.DataReader(v, 'yahoo', '2019-10-10', '2020-10-10')

United Arab Emirates
United Kingdom


KeyError: 'Date'

In [21]:
min_date = min(df['First Announcement Date'])
max_date = max(df['First Announcement Date'])

starting_date = str((first_date - dt.timedelta(280)).date())
ending_date = str((last_date + dt.timedelta(80)).date())

In [35]:
# ClosePrice gives different results from PriceClose...
# see https://community.developers.refinitiv.com/questions/57842/get-data-inconsistent-between-an-index-and-a-singl.html
# ClosePrice appears to be superior as it incorporates specific calendar holidays

# convert Ultimate Parent Id to List[str] and remove .0
identifiers = list(df['Ultimate Parent Id'].astype(
    str).str.split('.', expand=True)[0].unique())

stock_prices, err = ek.get_data(
    identifiers,
    ['TR.ClosePrice.date', 'TR.ClosePrice'],
    {'SDate': starting_date,
     'EDate': ending_date})

In [73]:
stock_prices

In [74]:
stock_prices = stock_prices.pivot(index='Instrument', columns='Date', values='Close Price').transpose()

In [24]:
web.DataReader(, 'yahoo', str((first_date - dt.timedelta(280)).date()),
               str((last_date + dt.timedelta(80)).date()))

Attributes    Adj Close                     Close                      High  \
Symbols           ^GSPC        ^AXJO        ^GSPC        ^AXJO        ^GSPC   
Date                                                                          
2011-07-21  1343.800049  4556.000000  1343.800049  4556.000000  1347.000000   
2011-07-22  1345.020020  4602.899902  1345.020020  4602.899902  1346.099976   
2011-07-25  1337.430054  4530.399902  1337.430054  4530.399902  1344.319946   
2011-07-26  1331.939941  4573.299805  1331.939941  4573.299805  1338.510010   
2011-07-27  1304.890015  4537.399902  1304.890015  4537.399902  1331.910034   
...                 ...          ...          ...          ...          ...   
2020-03-04  3130.120117  6395.700195  3130.120117  6395.700195  3130.969971   
2020-03-05  3023.939941  6216.200195  3023.939941  6216.200195  3083.040039   
2020-03-06  2972.370117          NaN  2972.370117          NaN  2985.929932   
2020-03-08          NaN  5760.600098          NaN  5760.600098          NaN   
2020-03-09  2746.560059  5939.600098  2746.560059  5939.600098  2863.889893   

Attributes                       Low                      Open               \
Symbols           ^AXJO        ^GSPC        ^AXJO        ^GSPC        ^AXJO   
Date                                                                          
2011-07-21  4576.100098  1325.650024  4551.299805  1325.650024  4551.299805   
2011-07-22  4612.200195  1336.949951  4565.299805  1343.800049  4565.299805   
2011-07-25  4595.399902  1331.089966  4529.200195  1344.319946  4595.299805   
2011-07-26  4582.700195  1329.589966  4536.200195  1337.390015  4536.299805   
2011-07-27  4570.500000  1303.489990  4536.600098  1331.910034  4564.600098   
...                 ...          ...          ...          ...          ...   
2020-03-04  6447.100098  3034.379883  6325.399902  3045.750000  6325.399902   
2020-03-05  6395.700195  2999.830078  6216.200195  3075.699951  6395.700195   
2020-03-06          NaN  2901.540039          NaN  2954.199951          NaN   
2020-03-08  6216.200195          NaN  5760.600098          NaN  6216.200195   
2020-03-09  5939.600098  2734.429932  5538.899902  2863.889893  5760.600098   

Attributes        Volume             
Symbols            ^GSPC      ^AXJO  
Date                                 
2011-07-21  4.837430e+09        0.0  
2011-07-22  3.522830e+09        0.0  
2011-07-25  3.536890e+09        0.0  
2011-07-26  4.007050e+09        0.0  
2011-07-27  3.479040e+09        0.0  
...                  ...        ...  
2020-03-04  5.035480e+09   989800.0  
2020-03-05  5.575550e+09   959400.0  
2020-03-06  6.552140e+09        NaN  
2020-03-08           NaN  1444300.0  
2020-03-09  8.423050e+09  1803100.0  

[2448 rows x 12 columns]

In [17]:
indexes, err = ek.get_data(
    '0#.SPX',
    ['TR.PriceClose.date', 'TR.PriceClose'],
    {'SDate': str((first_date - dt.timedelta(280)).date()),
     'EDate': str((last_date + dt.timedelta(80)).date())})

2020-10-18 18:01:06,554 P[12040] [MainThread 11876] UDF Core request failed. Gateway Time-out


EikonError: Error code 2504 | UDF Core request failed. Gateway Time-out

In [ ]:
indexes

In [83]:
data.columns

Index(['Instrument', 'Date', 'Price Close'], dtype='object')

In [ ]:
data = bonds.copy()
data['Date'] = pd.to_datetime(data['Date'])
#data.set_index('Date', inplace = True)

In [84]:
data.head()

,Instrument,Date,Price Close
0,4295864350,2011-07-21 00:00:00+00:00,10.291925
1,4295864350,2011-07-22 00:00:00+00:00,10.284924
2,4295864350,2011-07-25 00:00:00+00:00,9.815836
3,4295864350,2011-07-26 00:00:00+00:00,9.857844
4,4295864350,2011-07-27 00:00:00+00:00,9.934859


In [78]:
bonds

,Instrument,Date,Price Close
0,4295864350,2011-07-21T00:00:00Z,10.291925
1,4295864350,2011-07-22T00:00:00Z,10.284924
2,4295864350,2011-07-25T00:00:00Z,9.815836
3,4295864350,2011-07-26T00:00:00Z,9.857844
4,4295864350,2011-07-27T00:00:00Z,9.934859
...,...,...,...
400098,4296062372,2020-03-03T00:00:00Z,8.300000
400099,4296062372,2020-03-04T00:00:00Z,8.300000
400100,4296062372,2020-03-05T00:00:00Z,8.540000
400101,4296062372,2020-03-06T00:00:00Z,8.400000


In [36]:
b

Index(['2011-07-21T00:00:00Z', '2011-07-22T00:00:00Z', '2011-07-25T00:00:00Z',
       '2011-07-26T00:00:00Z', '2011-07-27T00:00:00Z', '2011-07-28T00:00:00Z',
       '2011-07-29T00:00:00Z', '2011-08-01T00:00:00Z', '2011-08-02T00:00:00Z',
       '2011-08-03T00:00:00Z',
       ...
       '2020-02-25T00:00:00Z', '2020-02-26T00:00:00Z', '2020-02-27T00:00:00Z',
       '2020-02-28T00:00:00Z', '2020-03-02T00:00:00Z', '2020-03-03T00:00:00Z',
       '2020-03-04T00:00:00Z', '2020-03-05T00:00:00Z', '2020-03-06T00:00:00Z',
       '2020-03-09T00:00:00Z'],
      dtype='object', name='Date', length=2187)

In [18]:
bonds.set_index('Date')

,Instrument,Price Close
Date,,
2011-07-21T00:00:00Z,4295864350,10.291925
2011-07-22T00:00:00Z,4295864350,10.284924
2011-07-25T00:00:00Z,4295864350,9.815836
2011-07-26T00:00:00Z,4295864350,9.857844
2011-07-27T00:00:00Z,4295864350,9.934859
...,...,...
2020-03-03T00:00:00Z,4296062372,8.300000
2020-03-04T00:00:00Z,4296062372,8.300000
2020-03-05T00:00:00Z,4296062372,8.540000


In [18]:
def get_stock_data(stock: str, announcement_date) -> None:
    """
    Download from Yahoo stock Close price in period [-220, 60] around announcement_date.
    :param stock: str of stock ticker (e.g.: AAPL)
    :return: None
    """
    start = announcement_date - dt.timedelta(420)
    end = announcement_date + dt.timedelta(120)
    try:
        df = web.DataReader(stock.upper(), 'yahoo', start, end)
    except Exception as e:
        print(stock, ':', e)
        return None
    
    # >= announcement_date might be a non-trading day, so get first day that is greater or equal to announcement.
    announcement_date_position = (df.index>=announcement_date).searchsorted(1)
    df = df.iloc[announcement_date_position-220: announcement_date_position+60]
    return df.Close

In [19]:
len(set(zip(df.Ticker, df['First Announcement Date'])))

1274

In [20]:
is_private = dict()
results = dict()

In [22]:
for ticker, announcement_date in tqdm(set(zip(df.Ticker, df['First Announcement Date']))):
    if is_private.get(ticker) is not True:
        results[(ticker, announcement_date)] = get_stock_data(ticker, announcement_date)
        
        if results[(ticker, announcement_date)] is None:
            is_private[ticker] = True
        else:
            print(ticker, 'found prices')
            is_private[ticker] = False

EAXX : No data fetched for symbol EAXX using YahooDailyReader
KOMBK : No data fetched for symbol KOMBK using YahooDailyReader
ZHGOVE : No data fetched for symbol ZHGOVE using YahooDailyReader
ENGIE : No data fetched for symbol ENGIE using YahooDailyReader
CABKO : No data fetched for symbol CABKO using YahooDailyReader
GETP : No data fetched for symbol GETP using YahooDailyReader
CAGRAF : No data fetched for symbol CAGRAF using YahooDailyReader
ELLKV : No data fetched for symbol ELLKV using YahooDailyReader
SUMFGI : No data fetched for symbol SUMFGI using YahooDailyReader
FRNYKR : No data fetched for symbol FRNYKR using YahooDailyReader
SMFRS : No data fetched for symbol SMFRS using YahooDailyReader
BNP : 'Date'
HSBABB : No data fetched for symbol HSBABB using YahooDailyReader
MNDEFA : No data fetched for symbol MNDEFA using YahooDailyReader
VWS : No data fetched for symbol VWS using YahooDailyReader
VATTE : No data fetched for symbol VATTE using YahooDailyReader
ZHDHQ : No data fetched

SASEPS : No data fetched for symbol SASEPS using YahooDailyReader
GRKOEO : No data fetched for symbol GRKOEO using YahooDailyReader
AKDHU : No data fetched for symbol AKDHU using YahooDailyReader
OSLOK : No data fetched for symbol OSLOK using YahooDailyReader
FORSAW : No data fetched for symbol FORSAW using YahooDailyReader
TRPLD : No data fetched for symbol TRPLD using YahooDailyReader
SUSTAF : No data fetched for symbol SUSTAF using YahooDailyReader
IBEFN : No data fetched for symbol IBEFN using YahooDailyReader
EBKG : No data fetched for symbol EBKG using YahooDailyReader
LVENG : No data fetched for symbol LVENG using YahooDailyReader
AZREO : No data fetched for symbol AZREO using YahooDailyReader
JSFLC : No data fetched for symbol JSFLC using YahooDailyReader
STERV : No data fetched for symbol STERV using YahooDailyReader
ORSTEW : No data fetched for symbol ORSTEW using YahooDailyReader
BRKMFC : No data fetched for symbol BRKMFC using YahooDailyReader
SPDEV : No data fetched for sy

HYMTRD : No data fetched for symbol HYMTRD using YahooDailyReader
ZUGN : No data fetched for symbol ZUGN using YahooDailyReader
GRWER : No data fetched for symbol GRWER using YahooDailyReader
ULVR : No data fetched for symbol ULVR using YahooDailyReader
QTC : No data fetched for symbol QTC using YahooDailyReader
SPRSK : No data fetched for symbol SPRSK using YahooDailyReader
OGIFIB : No data fetched for symbol OGIFIB using YahooDailyReader
MSFNL : No data fetched for symbol MSFNL using YahooDailyReader
RABO : No data fetched for symbol RABO using YahooDailyReader
ESNSA : No data fetched for symbol ESNSA using YahooDailyReader
HZUBK : No data fetched for symbol HZUBK using YahooDailyReader
KOMEFS : No data fetched for symbol KOMEFS using YahooDailyReader
MTRCNC : No data fetched for symbol MTRCNC using YahooDailyReader
ALPER : No data fetched for symbol ALPER using YahooDailyReader
HERFTW : No data fetched for symbol HERFTW using YahooDailyReader
BLGGD : No data fetched for symbol BLGGD

NAFIN : No data fetched for symbol NAFIN using YahooDailyReader
ASEDVB : No data fetched for symbol ASEDVB using YahooDailyReader
TWBSB : No data fetched for symbol TWBSB using YahooDailyReader
JRFIC : No data fetched for symbol JRFIC using YahooDailyReader
ALSSBM : No data fetched for symbol ALSSBM using YahooDailyReader
BARC : No data fetched for symbol BARC using YahooDailyReader
LAPST : No data fetched for symbol LAPST using YahooDailyReader
EBKGI : No data fetched for symbol EBKGI using YahooDailyReader
STEXE : No data fetched for symbol STEXE using YahooDailyReader
QINGDD : No data fetched for symbol QINGDD using YahooDailyReader
CHGQTO : No data fetched for symbol CHGQTO using YahooDailyReader
SEKRT : No data fetched for symbol SEKRT using YahooDailyReader
SOGNER : No data fetched for symbol SOGNER using YahooDailyReader
IDCOL : No data fetched for symbol IDCOL using YahooDailyReader
HUANAB : No data fetched for symbol HUANAB using YahooDailyReader
OPOSUY : No data fetched for s

CHNWD : No data fetched for symbol CHNWD using YahooDailyReader
JPEXC : No data fetched for symbol JPEXC using YahooDailyReader
PWFC : No data fetched for symbol PWFC using YahooDailyReader
ROSCG : No data fetched for symbol ROSCG using YahooDailyReader
CSDPR : No data fetched for symbol CSDPR using YahooDailyReader
NDXG : No data fetched for symbol NDXG using YahooDailyReader
CENTL : No data fetched for symbol CENTL using YahooDailyReader
TRNSL : No data fetched for symbol TRNSL using YahooDailyReader
CIPHL : No data fetched for symbol CIPHL using YahooDailyReader
AIGRAH : No data fetched for symbol AIGRAH using YahooDailyReader
XINJGQ : No data fetched for symbol XINJGQ using YahooDailyReader
TLINK : No data fetched for symbol TLINK using YahooDailyReader
CTBCHA : No data fetched for symbol CTBCHA using YahooDailyReader
GRTJ : No data fetched for symbol GRTJ using YahooDailyReader
XELPOW : No data fetched for symbol XELPOW using YahooDailyReader
KMUNK : No data fetched for symbol KMU

In [ ]:
results

In [12]:
set(zip(df.Ticker, df['First Announcement Date']))

{('EAXX', Timestamp('2019-07-25 00:00:00')),
 ('KOMBK', Timestamp('2015-05-18 00:00:00')),
 ('ZHGOVE', Timestamp('2019-11-28 00:00:00')),
 ('ENGIE', Timestamp('2018-01-10 00:00:00')),
 ('CABKO', Timestamp('2018-04-19 00:00:00')),
 ('GETP', Timestamp('2018-09-26 00:00:00')),
 ('CAGRAF', Timestamp('2019-02-12 00:00:00')),
 ('ELLKV', Timestamp('2019-12-05 00:00:00')),
 ('SUMFGI', Timestamp('2018-12-20 00:00:00')),
 ('FRNYKR', Timestamp('2019-05-02 00:00:00')),
 ('SMFRS', Timestamp('2018-09-11 00:00:00')),
 ('BNP', Timestamp('2019-05-14 00:00:00')),
 ('HSBABB', Timestamp('2017-11-21 00:00:00')),
 ('MNDEFA', Timestamp('2017-06-28 00:00:00')),
 ('VWS', Timestamp('2015-03-04 00:00:00')),
 ('VATTE', Timestamp('2019-06-13 00:00:00')),
 ('ZHDHQ', Timestamp('2019-11-20 00:00:00')),
 ('VASKNV', Timestamp('2018-08-27 00:00:00')),
 ('HRSEG', Timestamp('2016-12-20 00:00:00')),
 ('ATRLJ', Timestamp('2018-04-11 00:00:00')),
 ('JRCTT', Timestamp('2019-11-22 00:00:00')),
 ('CTDM', Timestamp('2019-01-09 0

In [16]:
a = get_stock_data('BNP', pd.to_datetime('2019-02-27'))

In [17]:
is_private['BNP']

True

# EIKON

In [32]:
symbolList = ["AMXL.MX", "CMXCPO.MX"]
ret = ek.get_timeseries(symbolList, start_date="2018-09-06T20:00:00", end_date="2018-09-06T20:10:00",
                        interval="tick", raw_output=True)

2020-10-17 15:24:54,870 P[2016] [MainThread 6384] Error with AMXL.MX: No data available for the requested date range
2020-10-17 15:24:54,870 P[2016] [MainThread 6384] Error with CMXCPO.MX: No data available for the requested date range
2020-10-17 15:24:54,870 P[2016] [MainThread 6384] AMXL.MX: No data available for the requested date range | CMXCPO.MX: No data available for the requested date range | 


EikonError: Error code -1 | AMXL.MX: No data available for the requested date range | CMXCPO.MX: No data available for the requested date range | 

In [33]:
ek.get_data(['VOD.L', 'GOOG.O'],
            ['TR.PriceClose.date','TR.PriceClose'],
           {'SDate':'24-Sep-2018', 'EDate':'29-Oct-2019'})

(    Instrument                  Date  Price Close
 0        VOD.L  2018-09-24T00:00:00Z       170.00
 1        VOD.L  2018-09-25T00:00:00Z       168.74
 2        VOD.L  2018-09-26T00:00:00Z       166.94
 3        VOD.L  2018-09-27T00:00:00Z       168.14
 4        VOD.L  2018-09-28T00:00:00Z       164.50
 ..         ...                   ...          ...
 551     GOOG.O  2019-10-23T00:00:00Z      1259.13
 552     GOOG.O  2019-10-24T00:00:00Z      1260.99
 553     GOOG.O  2019-10-25T00:00:00Z      1265.13
 554     GOOG.O  2019-10-28T00:00:00Z      1290.00
 555     GOOG.O  2019-10-29T00:00:00Z      1262.62
 
 [556 rows x 3 columns],
 None)

In [16]:
ek.get_timeseries(['VOD.L', 'GOOG.O'], 
                  start_date=timedelta(-5), 
                  end_date=timedelta(0), 
                  interval='day')

2020-10-16 20:28:39,524 P[13876] [MainThread 1836] Can not process metadata for request: TimeSeries.interval - value doesn't fall in valid range: [tick tas taq tastaq minute hour session daily weekly monthly quarterly yearly]


EikonError: Error code 1422 | Can not process metadata for request: TimeSeries.interval - value doesn't fall in valid range: [tick tas taq tastaq minute hour session daily weekly monthly quarterly yearly]

In [ ]:
error